# CMPS-320 Homework #4
## Classification Practice
Elijah Campbell-Ihim

11/13/23

# Section 1: Written Questions

1) If a model is great for the training data but is not generalizing well to new data, the model is likely overfitting.The variance is too high and the model is fitting too tightly to the training data. One solution is to use regularization, which will dampen the degree to which each feature weighs on the target prediction. Another solution is to collect much more data for the training data and feed it into the model. More training data means the model has seen more, and will be better equipped to understand and interpret new data effectively. If you look at the extreme case, if you had every bit of data imaginable, everything would be in the model’s training data, and it would accurately predict everything. 

<br/><br/>
2)  N1 = 5 (5 Fold CV)

    N2 = 160 (200 examples - 40 test data)
    
    N3 = 40 (200 examples / 5 fold)

<br/><br/>
3)  a) (TP + TN) / total = 985 / 1020 = 96.57 % accuracy

    b) (TP + FN) / total = 995 / 1020 = 97.55 % Majority class accuracy

    c) The classifier appears to be more useful than the majority-class baseline. While accuracy alone can be misleading with an unbalanced dataset such as this one, the combination of high precision and recall, along with a high F1-score, suggests that the classifier is performing well and is likely more useful than a majority-class baseline, especially in situations where correctly identifying positive instances is crucial.

    d) Precision = TP / TP + FP = 970 / (970 + 10) = .9898

    e) Recall = TP / TP + FN = 970 / (970 + 25) = .9749

    f) F1 = TP / (TP + (FN + FP)/2) = 970 / (970 + (10 + 25)/2) = .9823



# Section 2: Coding ML Problem

## Understanding the Dataset

In [129]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import warnings
warnings.filterwarnings('ignore')


%matplotlib inline
plt.style.use('seaborn-v0_8-white')

In [130]:
df = pd.read_csv("Caravan.csv")
df.head()

,Unnamed: 0,MOSTYPE,MAANTHUI,MGEMOMV,MGEMLEEF,MOSHOOFD,MGODRK,MGODPR,MGODOV,MGODGE,...,APERSONG,AGEZONG,AWAOREG,ABRAND,AZEILPL,APLEZIER,AFIETS,AINBOED,ABYSTAND,Purchase
0,1,33,1,3,2,8,0,5,1,3,...,0,0,0,1,0,0,0,0,0,No
1,2,37,1,2,2,8,1,4,1,4,...,0,0,0,1,0,0,0,0,0,No
2,3,37,1,2,2,8,0,4,2,4,...,0,0,0,1,0,0,0,0,0,No
3,4,9,1,3,3,3,2,3,2,4,...,0,0,0,1,0,0,0,0,0,No
4,5,40,1,4,2,10,1,4,1,4,...,0,0,0,1,0,0,0,0,0,No


In [131]:
df = df.drop(df.columns[0], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 86 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   MOSTYPE   5822 non-null   int64 
 1   MAANTHUI  5822 non-null   int64 
 2   MGEMOMV   5822 non-null   int64 
 3   MGEMLEEF  5822 non-null   int64 
 4   MOSHOOFD  5822 non-null   int64 
 5   MGODRK    5822 non-null   int64 
 6   MGODPR    5822 non-null   int64 
 7   MGODOV    5822 non-null   int64 
 8   MGODGE    5822 non-null   int64 
 9   MRELGE    5822 non-null   int64 
 10  MRELSA    5822 non-null   int64 
 11  MRELOV    5822 non-null   int64 
 12  MFALLEEN  5822 non-null   int64 
 13  MFGEKIND  5822 non-null   int64 
 14  MFWEKIND  5822 non-null   int64 
 15  MOPLHOOG  5822 non-null   int64 
 16  MOPLMIDD  5822 non-null   int64 
 17  MOPLLAAG  5822 non-null   int64 
 18  MBERHOOG  5822 non-null   int64 
 19  MBERZELF  5822 non-null   int64 
 20  MBERBOER  5822 non-null   int64 
 21  MBERMIDD  5822

In [132]:
#This is used to calculate percentage of Purchases
count_yes = 0
for entry in df["Purchase"]:
    if entry == "Yes":
        count_yes += 1
        
percentageYes = count_yes / len(df["Purchase"]) * 100

print("Number of 'yes' occurrences:", count_yes)
print(f'Percentage of purchased from total: {percentageYes: .3f} %')

Number of 'yes' occurrences: 348
Percentage of purchased from total:  5.977 %


#### 1) Understanding the Dataset: Given the provided datasets (as CSV files), load them and answer the following questions. <br>
##### (a) What is the dimension of the datasets?

5822 x 86

##### (b) How many predictors measure demographic characteristics?

43 Predictors measure demographics (M)

##### (c) What is the percentage of people who purchased caravan insurance

About 6% of people purchaed Caravan

## Data Preprocessing

In [133]:
def standardizeData(data):
    from sklearn.preprocessing import StandardScaler

    # Create a StandardScaler object
    scaler = StandardScaler()

    # Fit the scaler to data 
    scaler.fit(data)

    # Transform data using the scaler to standardize it, return as DataFrame
    return pd.DataFrame(scaler.transform(data))


In [134]:
#X = feature matrix, y = target variable, then we standardize X
X = df.drop("Purchase", axis = 1)

y = df["Purchase"]

column_names = X.columns.tolist()

X = standardizeData(X)

X.columns = column_names

X.describe()

,MOSTYPE,MAANTHUI,MGEMOMV,MGEMLEEF,MOSHOOFD,MGODRK,MGODPR,MGODOV,MGODGE,MRELGE,...,ALEVEN,APERSONG,AGEZONG,AWAOREG,ABRAND,AZEILPL,APLEZIER,AFIETS,AINBOED,ABYSTAND
count,5.822000e+03,5.822000e+03,5.822000e+03,5.822000e+03,5.822000e+03,5.822000e+03,5.822000e+03,5.822000e+03,5.822000e+03,5.822000e+03,...,5.822000e+03,5.822000e+03,5.822000e+03,5.822000e+03,5.822000e+03,5.822000e+03,5.822000e+03,5822.000000,5.822000e+03,5.822000e+03
mean,-7.749822e-17,-1.614038e-16,-2.099164e-16,2.154084e-16,-2.929067e-17,-1.220444e-17,7.932889e-17,-1.403511e-17,9.519466e-17,-1.476738e-16,...,-4.515644e-17,2.379867e-17,1.464533e-17,9.763555e-18,-1.183831e-16,-6.102222e-19,-1.830667e-18,0.000000,3.905422e-17,3.356222e-18
std,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,...,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086e+00,1.000086,1.000086e+00,1.000086e+00
min,-1.810219e+00,-2.725800e-01,-2.125697e+00,-2.444683e+00,-1.671134e+00,-6.943106e-01,-2.696827e+00,-1.051594e+00,-2.039739e+00,-3.238560e+00,...,-2.029099e-01,-7.316512e-02,-8.105460e-02,-5.992001e-02,-1.014358e+00,-2.270578e-02,-7.365027e-02,-0.150620,-8.734772e-02,-1.188165e-01
25%,-1.109590e+00,-2.725800e-01,-8.595001e-01,-1.216964e+00,-9.709796e-01,-6.943106e-01,-3.654101e-01,-1.051594e+00,-7.877902e-01,-6.198244e-01,...,-2.029099e-01,-7.316512e-02,-8.105460e-02,-5.992001e-02,-1.014358e+00,-2.270578e-02,-7.365027e-02,-0.150620,-8.734772e-02,-1.188165e-01
50%,4.473633e-01,-2.725800e-01,4.066966e-01,1.075466e-02,4.293284e-01,-6.943106e-01,2.174440e-01,-6.871064e-02,-1.618158e-01,-9.607727e-02,...,-2.029099e-01,-7.316512e-02,-8.105460e-02,-5.992001e-02,7.649707e-01,-2.270578e-02,-7.365027e-02,-0.150620,-8.734772e-02,-1.188165e-01
75%,8.366015e-01,-2.725800e-01,4.066966e-01,1.075466e-02,7.794054e-01,3.025516e-01,8.002982e-01,9.141723e-01,4.641586e-01,4.276698e-01,...,-2.029099e-01,-7.316512e-02,-8.105460e-02,-5.992001e-02,7.649707e-01,-2.270578e-02,-7.365027e-02,-0.150620,-8.734772e-02,-1.188165e-01
max,1.303687e+00,2.190544e+01,2.939090e+00,3.693911e+00,1.479559e+00,8.277450e+00,2.548861e+00,3.862821e+00,3.594031e+00,1.475164e+00,...,2.098707e+01,1.366772e+01,1.233736e+01,2.578114e+01,1.144094e+01,4.404165e+01,2.442874e+01,14.069579,2.202302e+01,1.654985e+01


In [135]:
#We define our testing and training sets in terms of X and y
X_test = X.head(1000)
X_train = X.tail(len(X) - 1000)
y_test = y.head(1000)
y_train = y.tail(len(y) - 1000)
print("X Test Set Shape:", X_test.shape)
print("X Training Set Shape:", X_train.shape)
print("y Test Set Shape:", y_test.shape)
print("y Training Set Shape:", y_train.shape)

X Test Set Shape: (1000, 85)
X Training Set Shape: (4822, 85)
y Test Set Shape: (1000,)
y Training Set Shape: (4822,)


In [136]:
#This is used to calculate number of Purchases
count_yes = 0
for entry in y_test:
    if entry == "Yes":
        count_yes += 1
print("Number of 'yes' occurrences:", count_yes)

Number of 'yes' occurrences: 59


#### 3) Split the datasets into a test set containing the first 1,000 observations and a training set, including the remaining observations.
##### (a) How many observations are in each set?

Test Set = 1000 observations, Training Set = 4822 observations

##### (b) How many customers purchased insurance in each set?

Test Set = 59 Purchased, Training Set = 289 Purchased

## Binary Classifier

In [137]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

def knn_metrics(k_value):
    knn = KNeighborsClassifier(n_neighbors=k_value)
    knn.fit(X_train, y_train)
    y_pred_knn = knn.predict(X_test.values)

    precision_knn = precision_score(y_test, y_pred_knn, pos_label = "Yes")
    recall_knn = recall_score(y_test, y_pred_knn, pos_label = "Yes")
    f1_knn = f1_score(y_test, y_pred_knn, pos_label='Yes')
    accuracy_knn = accuracy_score(y_test, y_pred_knn)

    print(f'K = {k_value}: Precision = {precision_knn:.3f}, Recall = {recall_knn:.3f}, F1 Score = {f1_knn: .3f}, Accuracy = {accuracy_knn: .3f}')


# Apply KNN for different values of K
k_values = [1, 3, 5]
for k in k_values:
    knn_metrics(k)

K = 1: Precision = 0.117, Recall = 0.153, F1 Score =  0.132, Accuracy =  0.882
K = 3: Precision = 0.200, Recall = 0.085, F1 Score =  0.119, Accuracy =  0.926
K = 5: Precision = 0.267, Recall = 0.068, F1 Score =  0.108, Accuracy =  0.934


The precision of the model is relatively low, but seems to be increasing as the value of k increases. Of the three, K = 5 seems to produce the best result in terms of precision, even though recall and f1 score are decreasing (as the difference between precision and recall is growing). Overall, this does not seem to be a very great model for the data. 

In [138]:
from sklearn.linear_model import SGDClassifier

# Create and fit the SGD classifier
sgd_classifier = SGDClassifier(random_state=42)
sgd_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred_SGD = sgd_classifier.predict(X_test)

# Calculate precision and recall
precision_SGD = precision_score(y_test, y_pred_SGD, pos_label='Yes')
recall_SGD = recall_score(y_test, y_pred_SGD, pos_label='Yes')
f1_SGD = f1_score(y_test, y_pred_SGD, pos_label='Yes')
accuracy_SGD = accuracy_score(y_test, y_pred_SGD)


print(f'Precision: {precision_SGD:.3f}')
print(f'Recall: {recall_SGD:.3f}')
print(f'F1 Score: {f1_SGD: .3f}')
print(f'Accuracy: {accuracy_SGD:.3f}')


Precision: 0.312
Recall: 0.085
F1 Score:  0.133
Accuracy: 0.935


#### Which classifier finds real patterns in the caravan dataset? The model that has the highest F1 score is the best.
The SDGClassifier model seems to be the best, as it has the highest F1 score and precision. It seems to have better captured trends in the data, however it has not an excellent model for the dataset. The F1, precision, and recall are all relatively low. In order to improve this model, we would have to better tune the hyperparameters or go back and improve the data (perhaps by feature engineering). Overall, the SGD model is the best one we have at predicting purchases, but could definitely be improved. 
